[Apatch Spark](https://spark.apache.org) は JVM 上に実装されたオープンソースの分散処理プログラミング環境で、[近年急速な盛り上がりを見せています](http://fortune.com/2015/09/25/apache-spark-survey/)。 今後 [pyspark](https://spark.apache.org/docs/latest/api/python/index.html) を通じて Spark を使う予定なので、試しに触ってみたいのですが、いかんせん JVM に慣れていないこともあって Spark のインストールから始めると大変です。

そこでこの記事では Docker を使って Spark と pyspark の環境を構築します。

## 環境を用意する

Jupyter Lab が公開している [jupyter/pyspark-notebook](https://hub.docker.com/r/jupyter/pyspark-notebook/) というズバリな Docker イメージがあるのでこれを使います。さしあたり現時点で公開されている最新版（`87210526f381`）を指定する docker-compose.yml を用意しました:

```yaml
version: "3"
services:
  notebook:
    image: jupyter/pyspark-notebook:87210526f381
    working_dir: /app/notebooks
    ports:
      - 8888:8888
    volumes:
      - ./notebooks:/app/notebooks
      - ./data:/app/data
```

2 つのディレクトリをマウントしています:

- `./notebooks` - Jupyter Notebook を保存するディレクトリ。作った Notebook を Docker の外に持ち出す目的でマウントしています。
- `./data` - 分析用のデータを格納するディレクトリです。今回の記事では使っていませんが、 CSV ファイルなどをここにおいて Jupyter から触れるようにします。

用意ができたら `docker-compse up` を実行して起動します:

```
$ docker-compose up
Starting pyspark-notebook_notebook_1 ... done
Attaching to pyspark-notebook_notebook_1
notebook_1  | Executing the command: jupyter notebook
notebook_1  | [I 10:19:29.742 NotebookApp] JupyterLab extension loaded from /opt/conda/lib/python3.6/site-packages/jupyterlab
notebook_1  | [I 10:19:29.743 NotebookApp] JupyterLab application directory is /opt/conda/share/jupyter/lab
notebook_1  | [I 10:19:29.745 NotebookApp] Serving notebooks from local directory: /app/notebooks
notebook_1  | [I 10:19:29.745 NotebookApp] The Jupyter Notebook is running at:
notebook_1  | [I 10:19:29.745 NotebookApp] http://(cbe66ae89e09 or 127.0.0.1):8888/?token=TOKEN
notebook_1  | [I 10:19:29.745 NotebookApp] Use Control-C to stop this server and shut down all kernels (twice to skip confirmation).
notebook_1  | [C 10:19:29.746 NotebookApp]
notebook_1  |
notebook_1  |     Copy/paste this URL into your browser when you connect for the first time,
notebook_1  |     to login with a token:
notebook_1  |         http://(HASH or 127.0.0.1):8888/?token=TOKEN
```

最後に表示された URL にアクセスすれば pyspark が使える Jupyter Notebook が表示されます。簡単ですね。インストールされている pyspark のバージョンは最新の "2.4.0" でした。

In [1]:
import pyspark
pyspark.version.__version__

'2.4.0'

ちなみにこの記事はまさにこうして起動した Jupyter Notebook を使って書かれています。

## Spark クラスタを起動する

Spark は通常クラスタを作って分散処理を行いますが、開発段階からクラスタを作るのは大変なので [local mode](https://jaceklaskowski.gitbooks.io/mastering-apache-spark/spark-local.html) が用意されています。

pyspark から local mode で Spark を起動します。`SparkContext` に渡している文字列は利用可能スレッド数を意味していて:

- `local` - 1 スレッドのみ使う
- `local[n]` - `n` スレッド使う（`n` は実際には数字が入る）
- `local[*]` - JVM で使えるプロセッサーの数だけスレッドを使う（内部では [`Runtime.getRuntime.availableProcessors()`](https://docs.oracle.com/javase/8/docs/api/java/lang/Runtime.html#availableProcessors--) を使っているらしい）

ということを表しています。一般的には `local[*]` が使われるようです。

In [2]:
sc = pyspark.SparkContext('local[*]')

試しに 0 から 10 までの数字の合計を計算してみます。

In [3]:
rdd = sc.parallelize(range(10))
rdd.sum()

45

使い終わったら停止します。

In [4]:
sc.stop()

## おわりに

この記事では Docker を使って pyspark 環境を作り、実際に Spark クラスタを起動してみました。Spark のことはまだまだ全然分かりませんが少しずつできることを増やしていこうと思います。

## 参考

- [jupyter/pyspark-notebook - Docker Hub](https://hub.docker.com/r/jupyter/pyspark-notebook/)
- [Image Specifics — docker-stacks latest documentation](https://jupyter-docker-stacks.readthedocs.io/en/latest/using/specifics.html#apache-spark)
- [pyspark package — PySpark 2.4.0 documentation](https://spark.apache.org/docs/latest/api/python/pyspark.html)
- [Get Started with PySpark and Jupyter Notebook in 3 Minutes](https://blog.sicara.com/get-started-pyspark-jupyter-guide-tutorial-ae2fe84f594f)
- [Spark local (pseudo-cluster) · Mastering Apache Spark](https://jaceklaskowski.gitbooks.io/mastering-apache-spark/spark-local.html)